# Feature Eng for Linear Regression Model

For linear regression model, the feature process work are:
- The one-hot encoding of caterogial features. The categorical features are pre-processed in the label encoding of Random Forest feature engineering process. The only work to do in the notebook is to one-hot-encoding the label encoded categorial faetures
- Bin the continous features into number of k groups

In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from sklearn import preprocessing, pipeline, metrics, model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import KBinsDiscretizer
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100
sns.set_style("white")
%matplotlib inline

In [3]:
# read input 
full_concat_df = pd.read_csv("df_to_RF_model.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (130,131,132,135,136,137,140) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#check 
full_concat_df[full_concat_df['table']=='a'].issue_d_year.value_counts()

2011    3232
Name: issue_d_year, dtype: int64

In [5]:
full_concat_df[full_concat_df['table']=='b'].issue_d_year.value_counts()

2013    3354
Name: issue_d_year, dtype: int64

### Encode loan_status as target

In [6]:
full_concat_df['target'] = full_concat_df['loan_status'].apply(lambda x: 1 if x == 'Charged Off' else 0)

In [7]:
full_concat_df['target'].value_counts()

0    5494
1    1092
Name: target, dtype: int64

In [8]:
encode_vars = ['tax_liens_le', 'home_ownership_le','verification_status_le','emp_length_impute_num', 
               'emp_title_truncated_impute_le','zip_code_truncated_le','term_encode', 'purpose_le']

num_vars = ['pub_rec_bankruptcies','dti','loan_amnt', 'annual_inc' ,'loan_income_ratio','fico_range_low','fico_range_high']





### Hot-hot encoding

In [9]:
dummies_encode_vars = []
for encode_var in encode_vars:
    dummies_encode_vars.append(pd.get_dummies(full_concat_df[encode_var], prefix = encode_var, prefix_sep = '_'))

In [10]:
pd_dummies_vars = pd.concat(dummies_encode_vars,axis=1)

In [12]:
len(pd_dummies_vars.columns)

153

In [22]:
full_concat_df[['pub_rec_bankruptcies']].value_counts()`

0    6140
1     434
2      10
3       1
4       1
Name: pub_rec_bankruptcies, dtype: int64

'pub_rec_bankruptcies' will be one hot encoder later

### Bin the continus features

In [50]:
# bin to N_BIN groups
N_BIN = 4
num_encode_vars = ['dti','loan_amnt', 'annual_inc' ,'loan_income_ratio','fico_range_low','fico_range_high']
bin_encode_vars = []
pd_bin_encode = pd.DataFrame()
bin_encode =  KBinsDiscretizer(n_bins=N_BIN, encode='onehot-dense')
for num_var in num_encode_vars:
    print(f"bin the var: {num_var}")
    col_names = []
    bin_encode_var = num_var + '_'+'bin_encode'
    for i in range(N_BIN):
        col_names.append(bin_encode_var+'_'+str(i+1)+'_'+'qrt')
    bin_encode_vars.append(pd.DataFrame(data =bin_encode.fit_transform(full_concat_df[[num_var]]),columns =col_names ))

bin the var: dti
bin the var: loan_amnt
bin the var: annual_inc
bin the var: loan_income_ratio
bin the var: fico_range_low
bin the var: fico_range_high


In [51]:
pd_bin_num_vars = pd.concat(bin_encode_vars,axis=1)

Encode the "pub_rec_bankruptcies"

In [56]:
pd_pub_bankrupt = pd.DataFrame()
pd_pub_bankrupt['pub_bankrupt_zero_record'] = np.where((full_concat_df['pub_rec_bankruptcies']==0),1,0)
pd_pub_bankrupt['pub_bankrupt_more_than_one_record'] = np.where((full_concat_df['pub_rec_bankruptcies']>0),1,0)

Concate the encoded features with aterget

In [67]:
pd_target = pd.DataFrame(data=full_concat_df['target'],columns=['target'])

In [79]:
pd_features_lr = pd.concat([pd_dummies_vars,pd_pub_bankrupt,pd_bin_num_vars,pd_target], axis=1)

In [80]:
pd_features_lr.head()

,tax_liens_le_0,tax_liens_le_1,tax_liens_le_2,tax_liens_le_3,tax_liens_le_4,tax_liens_le_5,home_ownership_le_0,home_ownership_le_1,home_ownership_le_2,verification_status_le_0,verification_status_le_1,verification_status_le_2,emp_length_impute_num_-999,emp_length_impute_num_1,emp_length_impute_num_2,emp_length_impute_num_3,emp_length_impute_num_4,emp_length_impute_num_5,emp_length_impute_num_6,emp_length_impute_num_7,emp_length_impute_num_8,emp_length_impute_num_9,emp_length_impute_num_10,emp_title_truncated_impute_le_0,emp_title_truncated_impute_le_1,emp_title_truncated_impute_le_2,emp_title_truncated_impute_le_3,emp_title_truncated_impute_le_4,emp_title_truncated_impute_le_5,emp_title_truncated_impute_le_6,emp_title_truncated_impute_le_7,emp_title_truncated_impute_le_8,emp_title_truncated_impute_le_9,emp_title_truncated_impute_le_10,emp_title_truncated_impute_le_11,emp_title_truncated_impute_le_12,emp_title_truncated_impute_le_13,emp_title_truncated_impute_le_14,emp_title_truncated_impute_le_15,emp_title_truncated_impute_le_16,emp_title_truncated_impute_le_17,emp_title_truncated_impute_le_18,emp_title_truncated_impute_le_19,zip_code_truncated_le_0,zip_code_truncated_le_1,zip_code_truncated_le_2,zip_code_truncated_le_3,zip_code_truncated_le_4,zip_code_truncated_le_5,zip_code_truncated_le_6,zip_code_truncated_le_7,zip_code_truncated_le_8,zip_code_truncated_le_9,zip_code_truncated_le_10,zip_code_truncated_le_11,zip_code_truncated_le_12,zip_code_truncated_le_13,zip_code_truncated_le_14,zip_code_truncated_le_15,zip_code_truncated_le_16,zip_code_truncated_le_17,zip_code_truncated_le_18,zip_code_truncated_le_19,zip_code_truncated_le_20,zip_code_truncated_le_21,zip_code_truncated_le_22,zip_code_truncated_le_23,zip_code_truncated_le_24,zip_code_truncated_le_25,zip_code_truncated_le_26,zip_code_truncated_le_27,zip_code_truncated_le_28,zip_code_truncated_le_29,zip_code_truncated_le_30,zip_code_truncated_le_31,zip_code_truncated_le_32,zip_code_truncated_le_33,zip_code_truncated_le_34,zip_code_truncated_le_35,zip_code_truncated_le_36,zip_code_truncated_le_37,zip_code_truncated_le_38,zip_code_truncated_le_39,zip_code_truncated_le_40,zip_code_truncated_le_41,zip_code_truncated_le_42,zip_code_truncated_le_43,zip_code_truncated_le_44,zip_code_truncated_le_45,zip_code_truncated_le_46,zip_code_truncated_le_47,zip_code_truncated_le_48,zip_code_truncated_le_49,zip_code_truncated_le_50,zip_code_truncated_le_51,zip_code_truncated_le_52,zip_code_truncated_le_53,zip_code_truncated_le_54,zip_code_truncated_le_55,zip_code_truncated_le_56,zip_code_truncated_le_57,zip_code_truncated_le_58,zip_code_truncated_le_59,zip_code_truncated_le_60,zip_code_truncated_le_61,zip_code_truncated_le_62,zip_code_truncated_le_63,zip_code_truncated_le_64,zip_code_truncated_le_65,zip_code_truncated_le_66,zip_code_truncated_le_67,zip_code_truncated_le_68,zip_code_truncated_le_69,zip_code_truncated_le_70,zip_code_truncated_le_71,zip_code_truncated_le_72,zip_code_truncated_le_73,zip_code_truncated_le_74,zip_code_truncated_le_75,zip_code_truncated_le_76,zip_code_truncated_le_77,zip_code_truncated_le_78,zip_code_truncated_le_79,zip_code_truncated_le_80,zip_code_truncated_le_81,zip_code_truncated_le_82,zip_code_truncated_le_83,zip_code_truncated_le_84,zip_code_truncated_le_85,zip_code_truncated_le_86,zip_code_truncated_le_87,zip_code_truncated_le_88,zip_code_truncated_le_89,zip_code_truncated_le_90,zip_code_truncated_le_91,zip_code_truncated_le_92,zip_code_truncated_le_93,zip_code_truncated_le_94,term_encode_0,term_encode_1,purpose_le_0,purpose_le_1,purpose_le_2,purpose_le_3,purpose_le_4,purpose_le_5,purpose_le_6,purpose_le_7,purpose_le_8,purpose_le_9,purpose_le_10,purpose_le_11,purpose_le_12,pub_bankrupt_zero_record,pub_bankrupt_more_than_one_record,dti_bin_encode_1_qrt,dti_bin_encode_2_qrt,dti_bin_encode_3_qrt,dti_bin_encode_4_qrt,loan_amnt_bin_encode_1_qrt,loan_amnt_bin_encode_2_qrt,loan_amnt_bin_encode_3_qrt,loan_amnt_bin_encode_4_qrt,annual_inc_bin_encode_1_qrt,annual_inc_b

In [81]:
len(pd_features_lr.columns)

180

In [82]:
# save to csv file 
pd_features_lr.to_csv("features_lr_model.csv")